# Part 1 - Data Ingestion

## Imports

In [2]:
import pandas as pd
import numpy as np
import spotipy as sp
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pprint

# Load environment variables from .env files which is named in the .gitignore file to prevent accidental upload to Github
import os
from dotenv import load_dotenv
load_dotenv()

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
name = "foo fighters"

## API call

In [3]:
results = spotify.search(q='artist:' + name, type='artist')
items = results['artists']['items']
artist_uri = ""
if len(items) > 0:
    artist = items[0]
    pprint.pprint(artist)
    artist_uri = artist["uri"]
    # print(artist['name'], artist['images'][0]['url'])

{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jy3rLJdDQY21OgRLCZ9sD'},
 'followers': {'href': None, 'total': 10197390},
 'genres': ['alternative metal',
            'alternative rock',
            'modern rock',
            'permanent wave',
            'post-grunge',
            'rock'],
 'href': 'https://api.spotify.com/v1/artists/7jy3rLJdDQY21OgRLCZ9sD',
 'id': '7jy3rLJdDQY21OgRLCZ9sD',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab6761610000e5eb9a43b87b50cd3d03544bb3e5',
             'width': 640},
            {'height': 320,
             'url': 'https://i.scdn.co/image/ab676161000051749a43b87b50cd3d03544bb3e5',
             'width': 320},
            {'height': 160,
             'url': 'https://i.scdn.co/image/ab6761610000f1789a43b87b50cd3d03544bb3e5',
             'width': 160}],
 'name': 'Foo Fighters',
 'popularity': 77,
 'type': 'artist',
 'uri': 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD'}


In [6]:
results = spotify.artist_albums(artist_id = artist_uri, album_type = 'album', country = 'US')
pprint.pprint(results['items'][0])
album_id = results['items'][0]['id']

{'album_group': 'album',
 'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3gmJYDVdrlGWWD5uVCUQOe'},
              'href': 'https://api.spotify.com/v1/artists/3gmJYDVdrlGWWD5uVCUQOe',
              'id': '3gmJYDVdrlGWWD5uVCUQOe',
              'name': 'Dream Widow',
              'type': 'artist',
              'uri': 'spotify:artist:3gmJYDVdrlGWWD5uVCUQOe'},
             {'external_urls': {'spotify': 'https://open.spotify.com/artist/7jy3rLJdDQY21OgRLCZ9sD'},
              'href': 'https://api.spotify.com/v1/artists/7jy3rLJdDQY21OgRLCZ9sD',
              'id': '7jy3rLJdDQY21OgRLCZ9sD',
              'name': 'Foo Fighters',
              'type': 'artist',
              'uri': 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD'}],
 'external_urls': {'spotify': 'https://open.spotify.com/album/2FfewmvnA0wctMD64KjOxP'},
 'href': 'https://api.spotify.com/v1/albums/2FfewmvnA0wctMD64KjOxP',
 'id': '2FfewmvnA0wctMD64KjOxP',
 'images': [{'height': 640,
     

In [5]:
results = spotify.album_tracks(album_id = album_id, limit=50, offset=0)
# pprint.pprint(results['items'][0])
track_id = results['items'][0]['id']

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3gmJYDVdrlGWWD5uVCUQOe'},
              'href': 'https://api.spotify.com/v1/artists/3gmJYDVdrlGWWD5uVCUQOe',
              'id': '3gmJYDVdrlGWWD5uVCUQOe',
              'name': 'Dream Widow',
              'type': 'artist',
              'uri': 'spotify:artist:3gmJYDVdrlGWWD5uVCUQOe'}],
 'available_markets': ['AD',
                       'AE',
                       'AG',
                       'AL',
                       'AM',
                       'AO',
                       'AR',
                       'AT',
                       'AU',
                       'AZ',
                       'BA',
                       'BB',
                       'BD',
                       'BE',
                       'BF',
                       'BG',
                       'BH',
                       'BI',
                       'BJ',
                       'BN',
                       'BO',
                       'BR'

In [2]:
results = spotify.audio_features([track_id])
pprint.pprint(results)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jy3rLJdDQY21OgRLCZ9sD'},
 'followers': {'href': None, 'total': 10197390},
 'genres': ['alternative metal',
            'alternative rock',
            'modern rock',
            'permanent wave',
            'post-grunge',
            'rock'],
 'href': 'https://api.spotify.com/v1/artists/7jy3rLJdDQY21OgRLCZ9sD',
 'id': '7jy3rLJdDQY21OgRLCZ9sD',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab6761610000e5eb9a43b87b50cd3d03544bb3e5',
             'width': 640},
            {'height': 320,
             'url': 'https://i.scdn.co/image/ab676161000051749a43b87b50cd3d03544bb3e5',
             'width': 320},
            {'height': 160,
             'url': 'https://i.scdn.co/image/ab6761610000f1789a43b87b50cd3d03544bb3e5',
             'width': 160}],
 'name': 'Foo Fighters',
 'popularity': 77,
 'type': 'artist',
 'uri': 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD'}
[{'acousticness': 3.55e-06,
  'analysis_url'